# Q-1. 
```
Imagine you have a dataset where you have different Instagram features
like u sername , Caption , Hashtag , Followers , Time_Since_posted , and likes , now your task is
to predict the number of likes and Time Since posted and the rest of the features are
your input features. Now you have to build a model which can predict the
number of likes and Time Since posted.
Dataset This is the Dataset You can use this dataset for this question.
```

In [75]:
import pandas as pd

In [76]:
df = pd.read_csv("instagram_reach (1).csv")

In [77]:
df.tail(5)

,Unnamed: 0,S.No,USERNAME,Caption,Followers,Hashtags,Time since posted,Likes
95,8,19,michaelgarza__,"328 S. Wetherly Drive, Beverly Hills, CA 90212...",614,#beverlyhills #realestate#losangelesrealestate...,3 hours,31
96,9,21,dvlp_search,Credit @tristankappel To find more dvlp follow...,450,#workspace #work #developer#development #devel...,3 hours,42
97,10,22,ecom.space,We are coming up with the Best 21 Books that w...,182,#books #book #motivation #inspiration #life#bo...,3 hours,10
98,11,24,lb3enterprises,We’re only paid to move dirt once. It’s not ju...,2039,#heavyequipment #underconstruction#dozer #real...,3 hours,222
99,12,25,palmariusdev,Obtén tu tienda en línea ahora.,741,#marketing #programming#development #desarroll...,3 hours,109


In [78]:
df["USERNAME"].nunique()

95

In [79]:
df["Time since posted"] = df["Time since posted"].apply(lambda x: x.split(" ")[0])

In [80]:
import numpy as np
df["Caption"] = df["Caption"].replace(np.nan, "")

In [81]:
df["Hashtags"] = df["Hashtags"].apply(lambda x: x.replace("#", " "))

In [82]:
df["Time since posted"] = df['Time since posted'].astype(int)

In [83]:
df["caption_length"] = df["Caption"].apply(lambda x: len(x.split(" ")))
df["hashtag_length"] = df["Hashtags"].apply(lambda x: len(x.split(" ")))

In [84]:
df.corr()

,Unnamed: 0,S.No,Followers,Time since posted,Likes,caption_length,hashtag_length
Unnamed: 0,1.000000,0.869163,0.208473,0.017372,0.010271,0.181740,0.174075
S.No,0.869163,1.000000,0.172002,-0.110024,0.009721,0.100464,0.118723
Followers,0.208473,0.172002,1.000000,0.251154,0.266218,-0.035585,0.000876
Time since posted,0.017372,-0.110024,0.251154,1.000000,0.609888,-0.017097,-0.095140
Likes,0.010271,0.009721,0.266218,0.609888,1.000000,-0.018454,-0.141778
caption_length,0.181740,0.100464,-0.035585,-0.017097,-0.018454,1.000000,0.194422
hashtag_length,0.174075,0.118723,0.000876,-0.095140,-0.141778,0.194422,1.000000


In [85]:
df["Time since posted"].unique()

array([11,  2,  3,  4,  7,  8,  9,  5, 20, 14, 24])

In [86]:
from nltk.corpus import stopwords  # To handle stopwords dictionary
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string  # to handle punctuations and special characters
import nltk

In [87]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/aneesaro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/aneesaro/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [88]:
# Define a function to map the part of speech tags from NLTK to WordNet
def get_wordnet_pos(word):
    """Map POS tag to first character used by WordNet lemmatizer"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()

def message_text_process(mess):
    mess = remove_word_with_digit_and_non_ascii(mess)
    # remove punctuation from string
    no_punctuation = [char for char in mess if char not in string.punctuation] 
    no_punctuation = ''.join(no_punctuation) # joining all the characters without punctuation
    # remove stop words
    words = [word.lower() for word in no_punctuation.split() if word not in stopwords.words('english')]
    words = " ".join(words)
    # lemmatize the text
    tokens = nltk.word_tokenize(words)
    lemmas = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, pos=get_wordnet_pos(token))
        lemmas.append(lemma)
    return lemmas

def remove_word_with_digit_and_non_ascii(sentence):
    final_sentence = ""
    sentence = ''.join(c for c in sentence if ord(c) < 128)
    for word in sentence:
        if not any(char.isdigit() for char in word):
            final_sentence += word
    return final_sentence

In [89]:
df["Caption"] = df["Caption"].apply(message_text_process)
df["Hashtags"] = df["Hashtags"].apply(message_text_process)

In [90]:
df["common_words"] = df.apply(lambda row: len(set(row['Caption']).intersection(row['Hashtags'])), axis=1)

In [91]:
X = df.drop(["Time since posted", "Likes", "Unnamed: 0", "S.No", "Caption", "Hashtags"], axis=1)
y = df[["Likes", "Time since posted"]]

In [92]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [93]:
import category_encoders as ce

cat_feat_to_encode = ['USERNAME']  # Specify the categorical variable to encode
username_encode = ce.CatBoostEncoder(cols=cat_feat_to_encode)  # avoids data leakage

X_train["USERNAME"] = username_encode.fit_transform(X_train["USERNAME"], y_train["Time since posted"])
X_test["USERNAME"] = username_encode.fit_transform(X_test["USERNAME"], y_test["Time since posted"])

In [94]:
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, HuberRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import HuberRegressor

In [95]:
models = {
    "linear_regression": LinearRegression(),
    "ridge_regression": Ridge(random_state=42),
    "lasso": Lasso(random_state=42),
    "etl": ExtraTreesRegressor(random_state=42, criterion="friedman_mse", n_estimators=51),
    "rf": RandomForestRegressor(random_state=42),
    "knn": KNeighborsRegressor(),    
}

r2_accuracy = []
trained_models_list = []
rmse_list = []

# looping through dictionary, create model and evaluates it
for model in list(models.values()):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_test_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2_accuracy.append(r2)
    rmse_list.append((rmse))
    trained_models_list.append(model)


In [96]:
r2_max_value = max(r2_accuracy)
r2_max_index = r2_accuracy.index(r2_max_value)
best_model = trained_models_list[r2_max_index]
best_model_name = list(models.keys())[r2_max_index]

In [97]:
print(f"{best_model_name}: {round(r2_max_value, 2)*100}")

etl: 48.0
